In [ ]:
pip install -U langchain langchain-openai

In [ ]:
pip install -U langchain-community

In [ ]:
pip install unstructured

In [ ]:
!pip install langchain_openai

In [ ]:
pip install "unstructured[md]"

In [5]:
import langchain
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import ChatPromptTemplate

In [18]:
def load_documents():
  loader = DirectoryLoader('books/')
  documents = loader.load()
  return documents

In [19]:
documents = load_documents()

In [20]:
from typing import List
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(documents: List[Document]):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500, length_function=len, add_start_index=True)
  chunks = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(chunks)} chunks.")


  return chunks

In [13]:
from openai import OpenAI

openai_api_key = "uiXn64-EBi__XOmb9n0W4xExkcT_iZBxFGhhHcsHKQ5YLrOs"
openai_api_base = "https://services.clarin-pl.eu/api/v1/oapi"


In [21]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = SentenceTransformerEmbeddings(model_name="nomic-ai/nomic-embed-text-v1", model_kwargs={"trust_remote_code":True})
vector_store = InMemoryVectorStore(embeddings)
splits = split_text(documents)
_ = vector_store.add_documents(documents=splits)

/root/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/eb02ceb48c1fdcc477ff1925c9732c379f0f0d1f/modeling_hf_nomic_bert.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this 

Split 1 documents into 120 chunks.


In [22]:

def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    sources = "\n\n".join(
        (f"Sources: {doc.metadata}")
        for doc in retrieved_docs
    )
    return serialized, sources


In [23]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""


In [24]:
def create_prompt(context_text, query_text):
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)
  return prompt


In [27]:
from langchain.chat_models import ChatOpenAI

client = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, model="llama")

def generate_response_with_rag(query):
  results = retrieve(query)
  context_text, sources = results
  prompt = create_prompt(context_text, query)
  response = client.predict(prompt)
  formatted_response = f"Response: {response}"
  return formatted_response

def generate_response(query):
  response = client.predict(query)
  formatted_response = f"Response: {response}"
  return formatted_response

In [29]:
import textwrap
def print_response(response):
  print(textwrap.fill(response, width=80))

In [31]:

query_text = "What flavour liquid did Alice drink?"
print_response(generate_response(query_text))
print()
print_response(generate_response_with_rag(query_text))

Response: In Lewis Carroll's 'Alice's Adventures in Wonderland', Alice drank a
bottle of 'Drink Me' liquid, which tasted like cherry tart, custard, pine-apple,
roast turkey, toffee, and buttered toast all mixed together.

Response: The flavour of the liquid Alice drank was a mix of cherry-tart,
custard, pineapple, roast turkey, toffee, and hot buttered toast.


In [32]:

query_text = "How was the white rabbit dressed?"
print_response(generate_response(query_text))
print()
print_response(generate_response_with_rag(query_text))

Response: In Lewis Carroll's 'Alice's Adventures in Wonderland', the White
Rabbit is described as wearing a waistcoat.

Response: The White Rabbit was "splendidly" dressed and was carrying a pair of
white kid gloves in one hand and a large fan in the other.
